#### Лабораторная работа № 3. Разработка нейросетевых функций. Операция Convolution Transpose
#### БВТ2003 Никитин Степан Романович

In [41]:
import torch
import torch.nn.functional as F
import numpy as np

In [42]:
class ConvTranspose:
    def __init__(self, in_channels: int, out_channels: int, kernel_size: tuple, transp_stride: int = 1, padding: int = 0, 
                 dilation: int = 1, groups: int = 1, bias: bool = True, padding_mode: str = 'zeros') -> None:
        """
        Инициализация параметров транспонированной свертки
        """
        self.in_channels = in_channels       # количество входных каналов
        self.out_channels = out_channels     # количество выходных каналов
        self.kernel_size = kernel_size       # размер ядра свертки в формате (высота, ширина)
        self.transp_stride = transp_stride   # шаг при транспонированной свертке
        self.padding = padding               # количество добавляемых нулей вокруг входных данных
        self.dilation = dilation             # расширение
        self.groups = groups                 # количество групп
        self.bias = bias                     # использовать ли параметр смещения
        self.padding_mode = padding_mode     # режим добавления нулей при паддинге ('zeros', 'reflect', 'replicate', 'circular')

        # инициализация обучаемых параметров
        if bias:
            self.bias = torch.nn.Parameter(torch.rand(out_channels))
        else:
            self.bias = None

        # проверка правил для свертки с группами
        assert in_channels % groups == 0
        assert out_channels % groups == 0

        # создание случайного фильтра
        if isinstance(kernel_size, tuple):
            self.filter = torch.nn.Parameter(torch.rand(out_channels, in_channels // groups, kernel_size[0], kernel_size[1]))
        elif isinstance(kernel_size, int):
            self.filter = torch.nn.Parameter(torch.rand(out_channels, in_channels // groups, kernel_size, kernel_size))
        else:
            raise ValueError("Неподдерживаемый тип kernel_size")

    def forward(self, input_m: torch.Tensor) -> torch.Tensor:
        """
        Производит транспонированную свертку на входных данных
        """
        # применение паддинга в зависимости от режима
        if self.padding_mode == 'zeros':
            input_m = F.pad(input_m, (self.padding, self.padding, self.padding, self.padding), mode='constant', value=0)
        elif self.padding_mode == 'reflect':
            input_m = F.pad(input_m, (self.padding, self.padding, self.padding, self.padding), mode='reflect')
        elif self.padding_mode == 'replicate':
            input_m = F.pad(input_m, (self.padding, self.padding, self.padding, self.padding), mode='replicate')
        elif self.padding_mode == 'circular':
            input_m = self.circular_pad(input_m, self.padding)
        else:
            raise ValueError("Неподдерживаемый padding_mode")

        # увеличение выборки входной матрицы
        upsampled_matr = F.interpolate(input_m, scale_factor=self.transp_stride, mode='nearest')

        # применение паддинга для увеличенной матрицы
        pad = (self.kernel_size[0] - 1, self.kernel_size[1] - 1)
        pad_matr = F.pad(upsampled_matr, (pad[1], pad[1], pad[0], pad[0]), mode='constant')

        # инвертирование пространственных размеров ядра для ConvTranspose2d
        filter_for_transpose = torch.flip(self.filter, [2, 3])

        # операция ConvTranspose2d
        out_tensor = F.conv2d(pad_matr, filter_for_transpose, bias=self.bias, stride=1, padding=0, dilation=self.dilation, groups=self.groups)

        return out_tensor

    def circular_pad(self, x: torch.Tensor, pad: int) -> torch.Tensor:
        """
        Производит круговое паддинг вокруг входных данных
        """
        batch_size, channels, height, width = x.shape
        padded_x = torch.zeros((batch_size, channels, height + 2 * pad, width + 2 * pad))
        padded_x[:, :, pad:pad+height, pad:pad+width] = x
        
        for i in range(pad):
            padded_x[:, :, i, pad:pad+width] = x[:, :, -1-i, :]
            padded_x[:, :, -1-i, pad:pad+width] = x[:, :, i, :]
            padded_x[:, :, :, i] = padded_x[:, :, :, -1-i]
            padded_x[:, :, :, -1-i] = padded_x[:, :, :, i]
            
        return padded_x

In [43]:
def test_conv_transpose():
    conv_transpose = ConvTranspose(in_channels=1, out_channels=1, kernel_size=(3, 3), transp_stride=2, padding=1, padding_mode='circular')
    x = torch.ones((1, 1, 4, 4))        # входные данные
    result = conv_transpose.forward(x)
    print("Тестовый пример:")
    print(f"Input: {x}\n")
    print(f"Output:{result}")

In [44]:
test_conv_transpose()

Тестовый пример:
Input: tensor([[[[1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.]]]])

Output:tensor([[[[0.9933, 0.9933, 1.0267, 1.2103, 1.2990, 1.2990, 1.2990, 1.2990,
           1.2990, 1.2990, 1.2655, 1.0819, 0.9933, 0.9933],
          [0.9933, 0.9933, 1.5068, 2.0829, 3.0248, 3.0248, 3.0248, 3.0248,
           3.0248, 3.0248, 2.5113, 1.9351, 0.9933, 0.9933],
          [0.9933, 0.9933, 1.6766, 3.0763, 4.9799, 4.9799, 4.9799, 4.9799,
           4.9799, 4.9799, 4.2965, 2.8968, 0.9933, 0.9933],
          [0.9933, 0.9933, 1.6766, 3.0763, 4.9799, 4.9799, 4.9799, 4.9799,
           4.9799, 4.9799, 4.2965, 2.8968, 0.9933, 0.9933],
          [0.9933, 0.9933, 1.6766, 3.0763, 4.9799, 4.9799, 4.9799, 4.9799,
           4.9799, 4.9799, 4.2965, 2.8968, 0.9933, 0.9933],
          [0.9933, 0.9933, 1.6766, 3.0763, 4.9799, 4.9799, 4.9799, 4.9799,
           4.9799, 4.9799, 4.2965, 2.8968, 0.9933, 0.9933],
          [0.9933, 0.9933, 1.6766, 3.0763, 

#### Дополнительное задание :
#### Необходимо реализовать алгоритм работы транспонированной свертки через алгоритм двумерной свертки, реализованный в первой лабораторной

In [55]:
import torch
import numpy as np
from typing import Union

In [56]:
class Conv2D:
    def __init__(self, in_channels: int, out_channels: int, kernel_size: Union[int, tuple], stride: Union[int, tuple] = 1) -> None:
        """
        Конструктор класса Conv2D
        """
        self.in_channels = in_channels     # количество входных каналов (каналов входного тензора)
        self.out_channels = out_channels   # количество выходных каналов (каналов результата свертки)
        self.kernel_size = kernel_size     # размер ядра свертки (предполагается квадратное ядро)
        self.stride = stride               # шаг свертки

        # преобразование kernel_size в кортеж, если он целое число
        if isinstance(kernel_size, int):
            kernel_size = (kernel_size, kernel_size)

        # инициализация весов (ядер свертки)
        self.weights = np.random.randn(out_channels, in_channels, *kernel_size)
        self.bias = np.zeros((out_channels,))

    def forward(self, input_tensor: np.ndarray) -> np.ndarray:
        """
        Метод для выполнения операции свертки
        """
        in_channels, input_height, input_width = input_tensor.shape
        kernel_size = self.kernel_size

        # преобразование kernel_size в кортеж, если он целое число
        if isinstance(kernel_size, int):
            kernel_size = (kernel_size, kernel_size)

        # проверка на корректные размеры входного тензора
        if input_height < kernel_size[0] or input_width < kernel_size[1]:
            raise ValueError("Неверный размер ядра для входного тензора")

        # вычисление размеров результата свертки
        if isinstance(self.stride, int):
            stride_h = stride_w = self.stride
        elif isinstance(self.stride, tuple) and len(self.stride) == 2:
            stride_h, stride_w = self.stride
        else:
            raise ValueError("Недопустимый формат шага. Должно быть целое число или кортеж из двух целых чисел")

        output_height = (input_height - kernel_size[0]) // stride_h + 1
        output_width = (input_width - kernel_size[1]) // stride_w + 1

        # инициализация результата свертки
        result = np.zeros((self.out_channels, output_height, output_width))

        # выполнение свертки
        for i in range(0, input_height - kernel_size[0] + 1, stride_h):
            for j in range(0, input_width - kernel_size[1] + 1, stride_w):
                # окно свертки
                window = input_tensor[:, i:i + kernel_size[0], j:j + kernel_size[1]]

                # вычисление свертки и добавление bias
                result[:, i // stride_h, j // stride_w] = np.sum(window * self.weights, axis=(1, 2, 3)) + self.bias

        return result

    def conv_transpose(self, input_tensor: np.ndarray) -> np.ndarray:
        """
        Производит транспонированную свертку на входном тензоре
        """
        # извлечение размеров входного тензора
        in_channels, input_height, input_width = input_tensor.shape
        kernel_size = self.kernel_size
        
        # проверка и преобразование формата размера ядра и шага
        if isinstance(kernel_size, int):
            kernel_size = (kernel_size, kernel_size)

        if isinstance(self.stride, int):
            stride_h = stride_w = self.stride
        elif isinstance(self.stride, tuple) and len(self.stride) == 2:
            stride_h, stride_w = self.stride
        else:
            raise ValueError("Недопустимый формат шага. Должен быть целым числом или кортежем из двух целых чисел")
        
        # вычисление размеров выходного тензора
        output_height = (input_height - 1) * stride_h + kernel_size[0]
        output_width = (input_width - 1) * stride_w + kernel_size[1]
        
        # инициализация выходного тензора
        result = np.zeros((self.in_channels, output_height, output_width))
        
        # проход по входному тензору и выполнение транспонированной свертки
        for i in range(0, input_height, stride_h):
            for j in range(0, input_width, stride_w):
                window = input_tensor[:, i:i + kernel_size[0], j:j + kernel_size[1]]
                result[:, i:i + kernel_size[0], j:j + kernel_size[1]] += np.sum(
                    window[:, np.newaxis, np.newaxis, :, :, np.newaxis] * self.weights[:, :, :, np.newaxis, np.newaxis, :],
                    axis=(0, 3, 4))

        return result

In [57]:
def run_transposed_conv_test(in_channels: int, out_channels: int, kernel_size: Union[int, tuple], stride: Union[int, tuple]) -> None:
    """
    Функция для выполнения теста трехмерной операции свертки
    """
    conv_layer = Conv2D(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride)

    input_tensor = np.random.randn(out_channels, 5, 5)

    result = conv_layer.conv_transpose(input_tensor)

    print(f"Тест транспонированной свертки с параметрами:")
    print(f"  - in_channels: {in_channels}")
    print(f"  - out_channels: {out_channels}")
    print(f"  - kernel_size: {kernel_size}")
    print(f"  - stride: {stride}")
    print(f"\nРезультат транспонированной свертки:\n\n {result}")
    print(f"\n{'-'*75}\n")

In [58]:
def test_transposed_conv_params() -> None:
    """
    Функция для тестирования различных параметров трехмерной операции свертки
    """
    # параметры тестирования для первой группы
    run_transposed_conv_test(in_channels=8, out_channels=20, kernel_size=3, stride=1)
    
    # параметры тестирования для второй группы
    run_transposed_conv_test(in_channels=6, out_channels=15, kernel_size=3, stride=1)
    
    # параметры тестирования для третьей группы
    run_transposed_conv_test(in_channels=4, out_channels=8, kernel_size=3, stride=1)

In [59]:
test_transposed_conv_params()

Тест транспонированной свертки с параметрами:
  - in_channels: 8
  - out_channels: 20
  - kernel_size: 3
  - stride: 1

Результат транспонированной свертки:

 [[[ -7.91917856 -20.40668248 -24.42276415 -18.91846362   0.56730836
    10.14284023   3.66401159]
  [-16.81672951  24.12334892 -32.79501279 -15.54394613   5.43545096
    33.92445646  12.19018012]
  [ -3.64234029 -20.17301981 -74.1694404  -40.25252187  14.88493514
    33.33619422  11.81911029]
  [ -5.11460573  -1.46949761 -46.48832286 -18.45783524  20.37442293
    20.66965232   0.98290011]
  [ 21.83814009  13.22187552 -26.49381343 -41.30458058 -17.08563907
    -7.45364615  -2.74166664]
  [ 16.01755588  19.86664753 -16.29337646 -14.80294778 -17.40365575
    -7.34300012  -4.25465145]
  [ 12.39810287  15.13127084  -3.79924865  -5.08711763  -5.00655745
    -2.25317361   2.68411813]]

 [[  6.32413607   0.45239095   4.64635233  14.44093404  26.58230079
    18.85345653   0.78065681]
  [-13.12386623 -17.87420802  13.76628055   7.19573796 